# Redes Neuronales

Vamos a enseñarle a un modelo a que vea una imagen con números y que sea capaz de identificarlos!

## Importamos librerías

Este paso puede dar bastantes problemas y puede provocar que sea necesario trabajar en Google Colab, pues a veces las librerías no se detectan correctamente, en concreto tensorflow, con lo cual si no funciona correctamente, simplemente hay que irse a Colab y pegar el código allí

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

## Cargamos imágenes con las que vamos a entrenar al modelo

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

## Visualizamos un ejemplo

In [ ]:
plt.imshow(train_images[7], cmap='gray', interpolation='bicubic')

In [ ]:
train_labels[7]

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
train_labels[7]

## Instanciamos un modelo y añadimos parámetros

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
])

In [ ]:
model.add(layers.Flatten())

In [ ]:
model.add(layers.Dense(64, activation='relu'))

In [ ]:
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer= "adam", loss="categorical_crossentropy", metrics=["accuracy"])

## Entrenamos modelo

In [ ]:
model.fit(train_images, train_labels, epochs=3, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
print(f"Test accuracy: {test_acc}")

## Guardamos modelo

In [ ]:
model.save("mnist_model.h5")

## Ponemos a prueba el modelo

In [ ]:
imagen = test_images[4].reshape(28, 28)
plt.imshow(imagen, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
test_images[4]

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

modelo_cargado = load_model("mnist_model.h5")

In [ ]:
nueva_imagen = test_images[4]
nueva_imagen = np.expand_dims(nueva_imagen, axis=0)

In [ ]:
prediccion = modelo_cargado.predict(nueva_imagen)
digito = np.argmax(prediccion)
print(f"El modelo predice que el número número es: {digito}")

## Ponemos a prueba con una interfaz

En vez de testear el modelo con una imagen distinta, vamos a crear una interfaz donde poder dibujar y testear la identificación del modelo

In [ ]:
!pip install netron
import netron
netron.start("mnist_model.h5")

Tras ejecutar el código siguiente, se abrirá una interfaz donde poder dibujar. Si no funciona, es posible que haya que ejecutarlo en un archivo.py, como el que se encuentra en el repo

In [ ]:
!pip install gradio==3.50.2
import gradio as gr
import tensorflow as tf
import numpy as np
 
modelo = tf.keras.models.load_model("mnist_model.h5")
 
def clasificar_imagenes(img):
    img = np.reshape(img, (1, 28, 28, 1)).astype("float32") / 255
    predicciones = modelo.predict(img)
    digito_predicho = np.argmax(predicciones)
    return str(digito_predicho)
 
interfaz = gr.Interface(fn=clasificar_imagenes , inputs="sketchpad", outputs="label")
interfaz.launch()